In [ ]:
# Using Dataframe for Min and Max Temp by Location
from google.colab import files
uploaded = files.upload()

Saving 1800.csv to 1800.csv


In [ ]:
# Installing Spark 3.0.1 with dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
print('JDK 8 installed')
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
print('Spark 3.0.1 installer downloaded')
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
print('Spark 3.0.1 downloaded')
!pip install -q findspark
print('findspark installed')
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
print('Environment Variables Set')
import findspark
findspark.init()
print('Install Complete')

JDK 8 installed
Spark 3.0.1 installer downloaded
Spark 3.0.1 downloaded
findspark installed
Environment Variables Set
Install Complete


In [ ]:
from pyspark.sql import SparkSession, functions as func
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

# Create Spark Session
spark = SparkSession.builder.appName('MinMaxTemp').getOrCreate()
# Create custom schema as no header in file (True = allow nulls)
schema = StructType([StructField('stationID',StringType(),True),
                    StructField('date',IntegerType(),True),
                    StructField('measure_type',StringType(),True),
                    StructField('temperature',FloatType(),True)])

# Read dataset with schema
df = spark.read.schema(schema).csv('1800.csv')
df.printSchema()

# Filter for Min Temp
minTemp = df.filter(df.measure_type == 'TMIN')
# Select stationID and temperature
stationTemp = minTemp.select('stationID','temperature')

# Find min temp per station
stationMinTemp = stationTemp.groupBy('stationID').min('temperature')
stationMinTemp.show()

# Collect results
results = stationMinTemp.collect()
for result in results:
  print(result[0] + '\t{:.2F}C'.format(result[1]))

root
 |-- stationID: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: float (nullable = true)

+-----------+----------------+
|  stationID|min(temperature)|
+-----------+----------------+
|ITE00100554|          -148.0|
|EZE00100082|          -135.0|
+-----------+----------------+

ITE00100554	-148.00C
EZE00100082	-135.00C
